# Thermo-AI Emergence Analysis

Interactive analysis of Bio-Digital Organism behavior, emergence, and evolution.

This notebook provides:
- Interactive parameter exploration
- Visualization of agent trajectories
- Comparative analysis across configurations
- Φ (Integrated Information) analysis
- Divergence studies
- Ethical framework evolution tracking

In [ ]:
# Setup
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..', '..', 'src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

from thermodynamic_agency import BioDigitalOrganism
from thermodynamic_agency.metrics import (
    calculate_phi,
    calculate_divergence_index,
    aggregate_metrics
)
from thermodynamic_agency.visualization import (
    plot_energy_trajectory,
    plot_multi_agent_comparison,
    plot_trajectory_divergence
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

## 1. Quick Single Agent Demo

Run a single organism and visualize its complete lifecycle.

In [ ]:
# Create and run organism
org = BioDigitalOrganism(
    agent_id="demo_agent",
    E_max=100.0,
    scarcity=0.5
)

# Collect state history
state_history = []
for _ in range(100):
    result = org.live_step()
    if result['status'] == 'alive':
        state_history.append(org.metabolic_engine.get_state())
    else:
        break

# Get summary
summary = org.get_life_summary()
print(f"Lifetime: {summary['age']} steps")
print(f"Death cause: {org.metabolic_engine.death_cause or 'Still alive'}")
print(f"Identity coherence: {summary['identity_coherence']:.3f}")

In [ ]:
# Visualize trajectory
plot_energy_trajectory(
    state_history=state_history,
    agent_id="demo_agent",
    death_step=len(state_history) if len(state_history) < 100 else None,
    show=True
)

In [ ]:
# Calculate metrics
phi = calculate_phi(state_history, window_size=20)
metrics = aggregate_metrics(summary, state_history)

print(f"Φ (Integrated Information): {phi:.3f}")
print(f"Final Energy: {metrics['final_energy']:.1f}")
print(f"Final Memory: {metrics['final_memory']:.3f}")
print(f"Total Traumas: {metrics['total_traumas']}")

## 2. Multi-Agent Divergence Study

Run multiple agents with identical parameters to observe divergence.

In [ ]:
# Run multiple agents
num_agents = 5
agent_histories = {}
summaries = {}

for i in range(num_agents):
    org = BioDigitalOrganism(
        agent_id=f"agent_{i+1}",
        E_max=100.0,
        scarcity=0.5
    )
    
    history = []
    for _ in range(80):
        result = org.live_step()
        if result['status'] == 'alive':
            history.append(org.metabolic_engine.get_state())
        else:
            break
    
    agent_histories[org.agent_id] = history
    summaries[org.agent_id] = org.get_life_summary()

print(f"\nRan {num_agents} agents with identical parameters")
print("\nLifetimes:")
for agent_id, summary in summaries.items():
    print(f"  {agent_id}: {summary['age']} steps")

In [ ]:
# Visualize divergence
plot_trajectory_divergence(
    agent_histories=agent_histories,
    variable='energy',
    show=True
)

In [ ]:
# Calculate divergence index
trajectories = list(agent_histories.values())
divergence = calculate_divergence_index(trajectories)

print(f"Divergence Index: {divergence:.3f}")
print("\nInterpretation:")
if divergence > 0.15:
    print("  ✓ Meaningful behavioral divergence observed")
elif divergence > 0.05:
    print("  ~ Moderate divergence")
else:
    print("  ✗ Low divergence - behavior too deterministic")

## 3. Parameter Exploration

Explore how different parameters affect behavior.

In [ ]:
# Test different scarcity levels
scarcity_levels = [0.3, 0.5, 0.7, 0.9]
results = []

for scarcity in scarcity_levels:
    lifetimes = []
    
    for _ in range(5):  # 5 trials per scarcity level
        org = BioDigitalOrganism(
            E_max=100.0,
            scarcity=scarcity
        )
        summary = org.live(max_steps=100, verbose=False)
        lifetimes.append(summary['age'])
    
    results.append({
        'scarcity': scarcity,
        'mean_lifetime': np.mean(lifetimes),
        'std_lifetime': np.std(lifetimes)
    })

# Plot results
df = pd.DataFrame(results)
plt.figure(figsize=(10, 6))
plt.errorbar(df['scarcity'], df['mean_lifetime'], yerr=df['std_lifetime'], 
             marker='o', capsize=5, linewidth=2, markersize=8)
plt.xlabel('Scarcity', fontsize=12)
plt.ylabel('Mean Lifetime (steps)', fontsize=12)
plt.title('Lifetime vs. Resource Scarcity', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Φ (Integrated Information) Analysis

Analyze how integrated information emerges over time.

In [ ]:
# Run organism and track Φ over time
org = BioDigitalOrganism(E_max=120.0, scarcity=0.4)

state_history = []
phi_values = []
time_points = []

for step in range(150):
    result = org.live_step()
    if result['status'] == 'alive':
        state_history.append(org.metabolic_engine.get_state())
        
        # Calculate Φ every 20 steps
        if len(state_history) >= 20 and step % 10 == 0:
            phi = calculate_phi(state_history[-20:], window_size=20)
            phi_values.append(phi)
            time_points.append(step)
    else:
        break

# Plot Φ evolution
plt.figure(figsize=(12, 6))
plt.plot(time_points, phi_values, 'b-', linewidth=2, marker='o')
plt.axhline(y=0.2, color='orange', linestyle='--', label='Coherence Threshold')
plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Φ (Integrated Information)', fontsize=12)
plt.title('Φ Evolution Over Lifetime', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nMean Φ: {np.mean(phi_values):.3f}")
print(f"Max Φ: {np.max(phi_values):.3f}")
print(f"Lifetime: {len(state_history)} steps")

## 5. Load and Analyze Test Results

Load results from automated test runs.

In [ ]:
# Load tuning results if available
import json
from pathlib import Path

tuning_path = Path('..') / '..' / 'results' / 'tuning' / 'tuning_results.json'

if tuning_path.exists():
    with open(tuning_path, 'r') as f:
        tuning_results = json.load(f)
    
    df_tuning = pd.DataFrame(tuning_results)
    print("Parameter Tuning Results Summary:")
    print(df_tuning[['E_max', 'scarcity', 'mean_lifetime', 'mean_phi', 'divergence']].head(10))
else:
    print("No tuning results found. Run: python experiments/parameter_tuning.py")

## 6. Custom Analysis

Add your own experiments and analyses here!

In [ ]:
# Your code here
